In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# export
import torch
from torch import nn
import torch.nn.functional as F
# from typing import Dict, List
# import enum
# from enum import Enum

In [3]:
class ImportanceMap(nn.Module):
    def __init__(self, use_map = True):
        super().__init__()
        self.use_map = use_map
    def forward(self, x: torch.tensor):
        if not self.use_map:
            return x
        # choose the last channel as the importance map
        importance_map = x[:,-1,...].unsqu
        nn.Parameter(importance_map)
        

SyntaxError: unexpected EOF while parsing (<ipython-input-3-40d7c076675d>, line 1)

In [13]:
x =torch.round(torch.rand([2,2,2,2])*10)
print(x)
print(x.shape)

tensor([[[[0., 5.],
          [1., 2.]],

         [[4., 9.],
          [7., 8.]]],


        [[[1., 5.],
          [0., 6.]],

         [[7., 3.],
          [9., 4.]]]])
torch.Size([2, 2, 2, 2])


In [16]:
mymap = x[:,-1,...]
mymap

tensor([[[4., 9.],
         [7., 8.]],

        [[7., 3.],
         [9., 4.]]])

# continue reading  autoencoder_imgcomp._get_heatmap3D